In [1]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline
from FeatureEngineering2 import FeatureEngineering2


%matplotlib inline

print("OK\n")

OK



In [2]:
def geometric_mean(x):
    return np.exp(np.log(x[x>0]).mean())

In [3]:
"""
params_base = {
            'boosting_type': 'gbdt'
            ,'objective': 'binary'
            ,'metric': 'auc'
            ,'verbosity': -1
            ,'learning_rate': 0.05
            ,'n_estimators': 100000
            ,'random_state':123
            ,'bagging_freq': 1   
}
"""
params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.02
    ,'max_bin':400
    ,'max_depth': -1
    ,'num_leaves': 30
    ,'min_child_samples': 70
    ,'subsample': 1.0
    ,'subsample_freq': 1
    ,'colsample_bytree': 0.05
    ,'min_split_gain': 0.5
    ,'reg_alpha': 0.0
    ,'reg_lambda': 100
    ,'n_estimators': 10000
}

def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "num_leaves": trial.suggest_int("num_leaves", 8, 256)
            ,"min_child_samples": trial.suggest_int("min_child_samples", 5, 200)
            ,"min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True)
            ,"feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0)
            ,"bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0)
            ,"lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e+2, log=True)
            ,"lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e+2, log=True)
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
                    ,early_stopping_rounds=100
                    ,verbose=0
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

In [4]:
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.02
    ,'max_bin':400
    ,'max_depth': -1
    ,'num_leaves': 30
    ,'min_child_samples': 70
    ,'subsample': 1.0
    ,'subsample_freq': 1
    ,'colsample_bytree': 0.05
    ,'min_split_gain': 0.5
    ,'reg_alpha': 0.0
    ,'reg_lambda': 100
    ,'n_estimators': 10000
}

In [5]:
baseline = Baseline('SK_ID_CURR')
modules = Modules('SK_ID_CURR')
FeatureEngineering2 = FeatureEngineering2('SK_ID_CURR')

In [6]:
df = pd.read_csv(r'./home-credit-default-risk/application_train.csv')
test_df = pd.read_csv(r'./home-credit-default-risk/application_test.csv')
df = df.append(test_df).reset_index()

docs = [_f for _f in df.columns if 'FLAG_DOC' in _f]
live = [_f for _f in df.columns if ('FLAG_' in _f) & ('FLAG_DOC' not in _f) & ('_FLAG_' not in _f)]

# general cleaning procedures
df = df[df['CODE_GENDER'] != 'XNA']
df = df[df['AMT_INCOME_TOTAL'] < 20000000] # remove a outlier 117M
# NaN values for DAYS_EMPLOYED: 365.243 -> nan
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True) # set null value
df['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True) # set null value

### CODE_GENDER, 
# Categorical features with Binary encode (0 or 1; two categories)
#for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
#    df[bin_feature], uniques = pd.factorize(df[bin_feature])

### NAME_INCOME_TYPE
#df['NAME_INCOME_TYPE'].replace(['Maternity leave'], ['Unemployed'], inplace=True)
#df['NAME_INCOME_TYPE'].replace(['Working','Commercial associate','Pensioner','State servant','Unemployed','Student','Businessman','Maternity leave'], [0, 1, 2, 3, 4, 5, 6, 7], inplace=True)
### NAME_FAMILY_STATUS
#df['NAME_FAMILY_STATUS'].replace('Unknown', 'Married', inplace=True)
#df['NAME_FAMILY_STATUS'].replace(['Married','Single / not married','Civil marriage','Separated','Widow'], [0, 1, 2, 3, 4], inplace=True)



######################
# 欠損値への対応
######################

# 性別（CODE_GENDER）欠損値（XNA）であった場合、最頻値（F）とする事とする
df.loc[df['CODE_GENDER'] == 'XNA', 'CODE_GENDER'] = 'F'

# 婚姻状況（NAME_FAMILY_STATUS）が不明（Unknown）であった場合、最頻値（Married）とする事とする
df.loc[df['NAME_FAMILY_STATUS'] == 'Unknown', 'NAME_FAMILY_STATUS'] = 'Married'

# 就労日数（DAYS_EMPLOYED）が365243人という方がいるが、異常値なので一旦、欠損値として扱う
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)

# -----------------------------2022/07/18-----------------------------#
##################################################################
# カテゴリ変数の整備
# （主に出現頻度が少ないカテゴリを整備していく）
##################################################################
# 収入区分（NAME_INCOME_TYPE）は全部で8区分あるが、うち4区分は件数が小さい（30件以下）ため、
# 近い意味合いのクラスに併合してしまうこととする。
df.loc[df['NAME_INCOME_TYPE'] == 'Businessman', 'NAME_INCOME_TYPE'] = 'Commercial associate'
df.loc[df['NAME_INCOME_TYPE'] == 'Maternity leave', 'NAME_INCOME_TYPE'] = 'Pensioner'
df.loc[df['NAME_INCOME_TYPE'] == 'Student', 'NAME_INCOME_TYPE'] = 'State servant'
df.loc[df['NAME_INCOME_TYPE'] == 'Unemployed', 'NAME_INCOME_TYPE'] = 'Pensioner'

# 組織区分（ORGANIZATION_TYPE）は区分が多すぎるため、
# 頻度の小さいものは併合して整理することとする。
df["ORGANIZATION_TYPE"] = np.where(df["ORGANIZATION_TYPE"].str.contains("Business Entity"), 
"Business_Entity", df["ORGANIZATION_TYPE"])
df["ORGANIZATION_TYPE"] = np.where(df["ORGANIZATION_TYPE"].str.contains("Industry"), 
"Industry", df["ORGANIZATION_TYPE"])
df["ORGANIZATION_TYPE"] = np.where(df["ORGANIZATION_TYPE"].str.contains("Trade"),
"Trade", df["ORGANIZATION_TYPE"])
df["ORGANIZATION_TYPE"] = np.where(df["ORGANIZATION_TYPE"].str.contains("Transport"),
"Transport", df["ORGANIZATION_TYPE"])
df["ORGANIZATION_TYPE"] = np.where(df["ORGANIZATION_TYPE"].isin(["School", "Kindergarten", "University"]),
"Education", df["ORGANIZATION_TYPE"])
df["ORGANIZATION_TYPE"] = np.where(df["ORGANIZATION_TYPE"].isin(["Emergency","Police", "Medicine","Goverment", "Postal", "Military", "Security Ministries", "Legal Services"]),
"Official", df["ORGANIZATION_TYPE"])
df["ORGANIZATION_TYPE"] = np.where(df["ORGANIZATION_TYPE"].isin(["Bank", "Insurance"]),
"Finance", df["ORGANIZATION_TYPE"])
df["ORGANIZATION_TYPE"] = np.where(df["ORGANIZATION_TYPE"].str.contains("Goverment"), 
"Government", df["ORGANIZATION_TYPE"])
df["ORGANIZATION_TYPE"] = np.where(df["ORGANIZATION_TYPE"].isin(["Realtor", "Housing"]),
"Realty", df["ORGANIZATION_TYPE"])
df["ORGANIZATION_TYPE"] = np.where(df["ORGANIZATION_TYPE"].isin(["Hotel", "Restaurant","Services"]),
"TourismFoodSector", df["ORGANIZATION_TYPE"])
df["ORGANIZATION_TYPE"] = np.where(df["ORGANIZATION_TYPE"].isin(["Cleaning","Electricity", "Telecom", "Mobile", "Advertising", "Religion", "Culture"]),
"Other", df["ORGANIZATION_TYPE"])

# 職業区分（OCCUPATION_TYPE）は区分が多すぎるため、
# 頻度の小さいものは併合して整理することとする。
df["OCCUPATION_TYPE"] = np.where(df["OCCUPATION_TYPE"].isin(["Low-skill Laborers", "Cooking staff", "Security staff", "Private service staff", "Cleaning staff", "Waiters/barmen staff"]),
"Low_skill_staff", df["OCCUPATION_TYPE"])
df["OCCUPATION_TYPE"] = np.where(df["OCCUPATION_TYPE"].isin(["IT staff", "High skill tech staff"]),
"High_skill_staff", df["OCCUPATION_TYPE"])
df["OCCUPATION_TYPE"] = np.where(df["OCCUPATION_TYPE"].isin(["Secretaries", "HR staff","Realty agents"]),
"Others", df["OCCUPATION_TYPE"])

# 申込様式（NAME_TYPE_SUITE）は細かい区分が存在するが、構成比1%を切るものは全て Rare としてまとめる。
# また、Spouse, partner は スペースが入っていて使いづらいので改名しておく。
tmp = df["NAME_TYPE_SUITE"].value_counts() / len(df)
rare_labels = tmp[tmp < 0.01].index
df["NAME_TYPE_SUITE"] = np.where(df["NAME_TYPE_SUITE"].isin(rare_labels), 'Rare', df["NAME_TYPE_SUITE"])
del tmp
gc.collect()
df["NAME_TYPE_SUITE"] = np.where(df["NAME_TYPE_SUITE"].str.contains("Spouse, partner"),
"Spouse_partner", df["NAME_TYPE_SUITE"])

# 学歴区分（NAME_EDUCATION_TYPE）のうち「Academic degree」は頻度が低いので、
# Higher education に併合して整理することとする。
# また、Secondary / secondary special は スペースが入っていて使いづらいので改名しておく。
df["NAME_EDUCATION_TYPE"] = np.where(df["NAME_EDUCATION_TYPE"] == "Academic degree",
"Higher education", df["NAME_EDUCATION_TYPE"])
df["NAME_EDUCATION_TYPE"] = np.where(df["NAME_EDUCATION_TYPE"].str.contains("Secondary / secondary special"),
"Secondary_secondary_special", df["ORGANIZATION_TYPE"])

# NAME_FAMILY_STATUS の Single / not married は スペースが入っていて使いづらいので改名しておく。
df["NAME_FAMILY_STATUS"] = np.where(df["NAME_FAMILY_STATUS"].str.contains("Single / not married"),
"Single_not_married", df["NAME_FAMILY_STATUS"])

# NAME_HOUSING_TYPE の House / apartment は スペースが入っていて使いづらいので改名しておく。
df["NAME_HOUSING_TYPE"] = np.where(df["NAME_HOUSING_TYPE"].str.contains("House / apartment"),
"House_apartment", df["NAME_HOUSING_TYPE"])

# 地域フラグ（REG_〇〇）についてはフラグが 1 となっているカラム数を総計した NEW_REGION という変数を作成した。
# 元の変数は削除する事とした。
cols = ["REG_REGION_NOT_LIVE_REGION","REG_REGION_NOT_WORK_REGION", "LIVE_REGION_NOT_WORK_REGION", 
"REG_CITY_NOT_LIVE_CITY","REG_CITY_NOT_WORK_CITY","LIVE_CITY_NOT_WORK_CITY"]
df["NEW_REGION"] = df[cols].sum(axis = 1)
df.drop(cols, axis = 1, inplace = True)

# 提出書類フラグ（FLAG_DOC_〇〇）についてはフラグが 1 となっているカラム数を総計した NEW_DOCUMENT という変数を作成した。
# 元の変数は削除する事とした。
docs = [col for col in df.columns if 'FLAG_DOC' in col]
df['NEW_DOCUMENT'] = df[docs].sum(axis=1)
df.drop(docs, axis = 1, inplace = True)

# 申請日時の情報（WEEKDAY_APPR_PROCESS_START と HOUR_APPR_PROCESS_START）については
# 元の変数は残しつつも、周期エンコーディングを実行した。
# （曜日・時刻を半径1の円周上に配置し、その座標を変数とする）
weekday_dict = {'MONDAY': 1, 'TUESDAY': 2, 'WEDNESDAY': 3, 'THURSDAY': 4, 'FRIDAY': 5, 'SATURDAY': 6, 'SUNDAY': 7}
df.replace({'WEEKDAY_APPR_PROCESS_START': weekday_dict}, inplace=True)
df['NEW_WEEKDAY_APPR_PROCESS_START' + "_SIN"] = np.sin(2 * np.pi * df["WEEKDAY_APPR_PROCESS_START"]/7)
df["NEW_WEEKDAY_APPR_PROCESS_START" + "_COS"] = np.cos(2 * np.pi * df["WEEKDAY_APPR_PROCESS_START"]/7)
df['NEW_HOUR_APPR_PROCESS_START' + "_SIN"] = np.sin(2 * np.pi * df["HOUR_APPR_PROCESS_START"]/23)
df["NEW_HOUR_APPR_PROCESS_START" + "_COS"] = np.cos(2 * np.pi * df["HOUR_APPR_PROCESS_START"]/23)
###----------------------------------2022/07/18-----------------------------------###






# nullに置き換え
df['DAYS_EMPLOYED'] = df['DAYS_EMPLOYED'].replace(365243, np.nan)

inc_by_org = df[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']

# 特徴量1: 総所得金額を世帯人数で除した値
df['INCOME_div_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']

# 特徴量2: 総所得額を就労期間で除した値
df['INCOME_div_EMPLOYED'] = df['AMT_INCOME_TOTAL'] / df['DAYS_EMPLOYED']

#
df['NEW_CREDIT_TO_ANNUITY_RATIO'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']

df['NEW_CREDIT_TO_GOODS_RATIO'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']

#df['NEW_DOC_IND_KURT'] = df[docs].kurtosis(axis=1)

#df['NEW_LIVE_IND_SUM'] = df[live].sum(axis=1)

df['NEW_INC_PER_CHLD'] = df['AMT_INCOME_TOTAL'] / (1 + df['CNT_CHILDREN'])

df['NEW_INC_BY_ORG'] = df['ORGANIZATION_TYPE'].map(inc_by_org)

df['NEW_EMPLOY_TO_BIRTH_RATIO'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']

df['NEW_EMPLOY_TO_BIRTH-18_RATIO'] = df['DAYS_EMPLOYED'] / (df['DAYS_BIRTH'] + 18*365)

df['NEW_BIRTH_TO_EMPLOY_RATIO'] = df['DAYS_BIRTH'] / (1 + df['DAYS_EMPLOYED'])

df['NEW_INCOME_TO_ANNUITY_RATIO'] = df['AMT_INCOME_TOTAL'] / (1 + df['AMT_ANNUITY'])

df['NEW_ANNUITY_TO_INCOME_RATIO'] = df['AMT_ANNUITY'] / (1 + df['AMT_INCOME_TOTAL'])


# 特徴量3: 外部スコアの平均
df['NEW_EXT_SOURCES_MEDIAN'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].median(axis=1, skipna=True)

df['EXT_SOURCE_mean'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)

df['EXT_SOURCE_max'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].max(axis=1)

df['EXT_SOURCE_min'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].min(axis=1)

df['EXT_SOURCE_std'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)

df['EXT_SOURCE_count'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].notnull().sum(axis=1)

df['NEW_EXT_SOURCES_PROD'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].prod(axis=1, skipna=True, min_count=1)

df['NEW_EXT_SOURCES_MAD'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mad(axis=1, skipna=True)

df['NEW_EXT_SOURCES_GEO'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].apply(geometric_mean, axis=1)

# 特徴量4: 就労期間を年齢で除した値
df['DAYS_EMPLOYED_div_BERTH'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']

# 特徴量5: 年金支払金額を所得金額で除した値
df['ANNUITY_div_INCOME'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']

# 特徴量6: 年金支払額を借入金で除した値
df['ANNUITY_div_CREDIT'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']

### 
df['credit_goods_price_ratio'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']
 
df['credit_downpayment'] = df['AMT_GOODS_PRICE'] - df['AMT_CREDIT']

df['USER_AGE'] = abs(df['DAYS_BIRTH'])/364.25

df['NEW_CAR_TO_BIRTH_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']

df['NEW_CAR_TO_EMPLOY_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']

df['NEW_PHONE_TO_BIRTH_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']

df['NEW_PHONE_TO_EMPLOYED_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_EMPLOYED']

df['NEW_CREDIT_TO_INCOME_RATIO'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']

df['NEW_INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']

df['NEW_INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']

### 2022/06/25
df['app_EXT_SOURCE_prod'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
df['app_EXT_SOURCE_1_times_EXT_SOURCE_2'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2']
df['app_EXT_SOURCE_1_times_EXT_SOURCE_3'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_3']
df['app_EXT_SOURCE_2_times_EXT_SOURCE_3'] = df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
df['app_EXT_SOURCE_1_times_DAYS_EMPLOYED'] = df['EXT_SOURCE_1'] * df['DAYS_EMPLOYED']
df['app_EXT_SOURCE_2_times_DAYS_EMPLOYED'] = df['EXT_SOURCE_2'] * df['DAYS_EMPLOYED']
df['app_EXT_SOURCE_3_times_DAYS_EMPLOYED'] = df['EXT_SOURCE_3'] * df['DAYS_EMPLOYED']
df['app_EXT_SOURCE_1_PER_DAYS_BIRTH'] = df['EXT_SOURCE_1'] / df['DAYS_BIRTH']
df['app_EXT_SOURCE_2_PER_DAYS_BIRTH'] = df['EXT_SOURCE_2'] / df['DAYS_BIRTH']
df['app_EXT_SOURCE_3_PER_DAYS_BIRTH'] = df['EXT_SOURCE_3'] / df['DAYS_BIRTH']

### 2022/06/26
df['app_most_popular_AMT_GOODS_PRICE'] = df['AMT_GOODS_PRICE'] \
    .isin([225000, 450000, 675000, 900000]).map({True: 1, False: 0})
df['app_popular_AMT_GOODS_PRICE'] = df['AMT_GOODS_PRICE'] \
    .isin([1125000, 1350000, 1575000, 1800000, 2250000]).map({True: 1, False: 0})
df['app_OWN_CAR_AGE_div_DAYS_BIRTH'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']
df['app_OWN_CAR_AGE_div_DAYS_EMPLOYED'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']
df['app_DAYS_LAST_PHONE_CHANGE_div_DAYS_BIRTH'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']
df['app_DAYS_LAST_PHONE_CHANGE_div_DAYS_EMPLOYED'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_EMPLOYED']
df['app_DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] - df['DAYS_BIRTH']
df['app_DAYS_EMPLOYED_div_DAYS_BIRTH'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
df['app_CNT_CHILDREN_div_CNT_FAM_MEMBERS'] = df['CNT_CHILDREN'] / df['CNT_FAM_MEMBERS']

### 2022/06/29
#### EXT_SOURCE_X FEATURE
df['APP_SCORE1_TO_BIRTH_RATIO'] = df['EXT_SOURCE_1'] / (df['DAYS_BIRTH'] / 365.25)
df['APP_SCORE2_TO_BIRTH_RATIO'] = df['EXT_SOURCE_2'] / (df['DAYS_BIRTH'] / 365.25)
df['APP_SCORE3_TO_BIRTH_RATIO'] = df['EXT_SOURCE_3'] / (df['DAYS_BIRTH'] / 365.25)
df['APP_SCORE1_TO_EMPLOY_RATIO'] = df['EXT_SOURCE_1'] / (df['DAYS_EMPLOYED'] / 365.25)
#### Time ratios
df['ID_TO_BIRTH_RATIO'] = df['DAYS_ID_PUBLISH'] / df['DAYS_BIRTH']

### one-hotencoding
encoding_columns = []

#### one-hotencoding処理対象列を取得
for col in df.columns:
    if df[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

#### one-hotencoding処理対象列を追加
#encoding_columns.append('CODE_GENDER')
#encoding_columns.append('FLAG_OWN_CAR')
#encoding_columns.append('FLAG_OWN_REALTY')
#encoding_columns.append('NAME_INCOME_TYPE')
#encoding_columns.append('NAME_FAMILY_STATUS')

#### one-hotencoding実行
for col in encoding_columns:
    df = pd.get_dummies(df, columns=[col], dummy_na=True)

### 列名の不正な文字を置換(ドット対応)
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x))

### 極大値、極小値をNULLへ置換
df = df.replace([np.inf, -np.inf], np.nan)

len(df.columns)

df.head()

NAME_CONTRACT_TYPE
CODE_GENDER
FLAG_OWN_CAR
FLAG_OWN_REALTY
NAME_TYPE_SUITE
NAME_INCOME_TYPE
NAME_EDUCATION_TYPE
NAME_FAMILY_STATUS
NAME_HOUSING_TYPE
OCCUPATION_TYPE
ORGANIZATION_TYPE
FONDKAPREMONT_MODE
HOUSETYPE_MODE
WALLSMATERIAL_MODE
EMERGENCYSTATE_MODE


,index,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,WALLSMATERIAL_MODE_Stone_brick,WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_nan
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,...,0,0,0,0,1,0,0,1,0,0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,...,0,0,0,0,0,0,0,1,0,0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,...,0,0,0,0,0,0,1,0,0,1
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,...,0,0,0,0,0,0,1,0,0,1
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,...,0,0,0,0,0,0,1,0,0,1


In [7]:
### 列名から不正な文字を削除
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x))

### 列名から不正な文字を削除
for col in df.columns:
    df[col] = df[col].replace('\s', '_', regex=True)
    df[col] = df[col].replace(':', '_', regex=True)
    df[col] = df[col].replace('/', '_', regex=True)

df.head()

,index,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,WALLSMATERIAL_MODE_Stone_brick,WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_nan
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,...,0,0,0,0,1,0,0,1,0,0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,...,0,0,0,0,0,0,0,1,0,0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,...,0,0,0,0,0,0,1,0,0,1
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,...,0,0,0,0,0,0,1,0,0,1
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,...,0,0,0,0,0,0,1,0,0,1


In [8]:
bureau_origin = pd.read_csv('./home-credit-default-risk/bureau.csv')
bureau_balance_origin = pd.read_csv('./home-credit-default-risk/bureau_balance.csv')
pos_cash_balance_origin = pd.read_csv('./home-credit-default-risk/POS_CASH_balance.csv')
credit_card_balance_origin = pd.read_csv('./home-credit-default-risk/credit_card_balance.csv')
previous_application_origin = pd.read_csv('./home-credit-default-risk/previous_application.csv')
installments_payments_origin = pd.read_csv('./home-credit-default-risk/installments_payments.csv')
pos_cash_balance_origin.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [9]:
bureau = bureau_origin.copy()
bureau_balance = bureau_balance_origin.copy()
pos_cash_balance = pos_cash_balance_origin.copy()
credit_card_balance = credit_card_balance_origin.copy()
previous_application = previous_application_origin.copy()
installments_payments = installments_payments_origin.copy()
pos_cash_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [10]:
pos = pos_cash_balance.copy()
pos = baseline.reduce_mem_usage(pos)
print(pos.shape)
pos.head()

Memory usage of dataframe is 610.43 MB
Memory usage of optimization is 290039510.00 MB
Decreased by -47513511.5%
(10001358, 8)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [11]:
pos_ohe = pd.get_dummies(pos, columns=['NAME_CONTRACT_STATUS'], dummy_na=True)
col_ohe = sorted(list(set(pos_ohe.columns) - set(pos.columns)))
print(len(col_ohe))
col_ohe

10


['NAME_CONTRACT_STATUS_Active',
 'NAME_CONTRACT_STATUS_Amortized debt',
 'NAME_CONTRACT_STATUS_Approved',
 'NAME_CONTRACT_STATUS_Canceled',
 'NAME_CONTRACT_STATUS_Completed',
 'NAME_CONTRACT_STATUS_Demand',
 'NAME_CONTRACT_STATUS_Returned to the store',
 'NAME_CONTRACT_STATUS_Signed',
 'NAME_CONTRACT_STATUS_XNA',
 'NAME_CONTRACT_STATUS_nan']

In [12]:
pos_ohe_agg = pos_ohe.groupby('SK_ID_CURR').agg({
    "MONTHS_BALANCE": ["mean", "std", "min", "max"]
    ,"CNT_INSTALMENT": ["mean", "std", "min", "max"]
    ,"CNT_INSTALMENT_FUTURE": ["mean", "std", "min", "max"]
    ,"SK_DPD": ["mean", "std", "min", "max"]
    ,"SK_DPD_DEF": ["mean", "std", "min", "max"]
    # カテゴリ変数をone-hot-encodingした値の集約
    ,"NAME_CONTRACT_STATUS_Active": ["mean"]
    ,"NAME_CONTRACT_STATUS_Amortized debt": ["mean"]
    ,"NAME_CONTRACT_STATUS_Approved": ["mean"]
    ,"NAME_CONTRACT_STATUS_Canceled": ["mean"]
    ,"NAME_CONTRACT_STATUS_Completed": ["mean"]
    ,"NAME_CONTRACT_STATUS_Demand": ["mean"]
    ,"NAME_CONTRACT_STATUS_Returned to the store": ["mean"]
    ,"NAME_CONTRACT_STATUS_Signed": ["mean"]
    ,"NAME_CONTRACT_STATUS_XNA": ["mean"]
    ,"NAME_CONTRACT_STATUS_nan": ["mean"]
    # IDのユニーク数をカウント
    ,"SK_ID_PREV":["count", "nunique"]
})

# カラム名の付与
pos_ohe_agg.columns = [i + "_" + j for i,j in pos_ohe_agg.columns]
pos_ohe_agg = pos_ohe_agg.reset_index(drop=False)

print(pos_ohe_agg.shape)
pos_ohe_agg.head()

(337252, 33)


,SK_ID_CURR,MONTHS_BALANCE_mean,MONTHS_BALANCE_std,MONTHS_BALANCE_min,MONTHS_BALANCE_max,CNT_INSTALMENT_mean,CNT_INSTALMENT_std,CNT_INSTALMENT_min,CNT_INSTALMENT_max,CNT_INSTALMENT_FUTURE_mean,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100001,-72.555556,20.863312,-96,-53,4.000000,0.000000,4.0,4.0,1.444444,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9,2
1,100002,-10.000000,5.627314,-19,-1,24.000000,0.000000,24.0,24.0,15.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,19,1
2,100003,-43.785714,24.640162,-77,-18,10.107142,2.806597,6.0,12.0,5.785714,...,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0,0.0,28,3
3,100004,-25.500000,1.290994,-27,-24,3.750000,0.500000,3.0,4.0,2.250000,...,0.0,0.0,0.250000,0.0,0.0,0.000000,0.0,0.0,4,1
4,100005,-20.000000,3.316625,-25,-15,11.700000,0.948683,9.0,12.0,7.200000,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11,1


In [13]:
credit = credit_card_balance_origin.copy()
cerdit = baseline.reduce_mem_usage(credit)
print(credit.shape)
credit.head()

Memory usage of dataframe is 673.88 MB
Memory usage of optimization is 334107272.00 MB
Decreased by -49579318.7%
(3840312, 23)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970001,135000,0.0,877.5,0.0,877.5,1700.324951,...,0.000000,0.000000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.554688,45000,2250.0,2250.0,0.0,0.0,2250.000000,...,64875.554688,64875.554688,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.224609,450000,0.0,0.0,0.0,0.0,2250.000000,...,31460.085938,31460.085938,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.109375,225000,2250.0,2250.0,0.0,0.0,11795.759766,...,233048.968750,233048.968750,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.468750,450000,0.0,11547.0,0.0,11547.0,22924.890625,...,453919.468750,453919.468750,0.0,1,0.0,1.0,101.0,Active,0,0


In [14]:
# one-hotencoding(処理対象カラムを取得)
encoding_columns = []

for col in credit.columns:
    if credit[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

encoding_columns

NAME_CONTRACT_STATUS


['NAME_CONTRACT_STATUS']

In [15]:
# one-hot-encoding
for col in encoding_columns:
    credit_ohe = pd.get_dummies(credit, columns=[col], dummy_na=True)
    col_ohe = sorted(list(set(credit_ohe.columns) - set(credit.columns)))

print(len(col_ohe))
col_ohe

8


['NAME_CONTRACT_STATUS_Active',
 'NAME_CONTRACT_STATUS_Approved',
 'NAME_CONTRACT_STATUS_Completed',
 'NAME_CONTRACT_STATUS_Demand',
 'NAME_CONTRACT_STATUS_Refused',
 'NAME_CONTRACT_STATUS_Sent proposal',
 'NAME_CONTRACT_STATUS_Signed',
 'NAME_CONTRACT_STATUS_nan']

In [16]:
credit_ohe_agg = credit_ohe.groupby('SK_ID_CURR').agg(
    {
        # 数値変数の集約
        'SK_ID_CURR': ['mean', 'std', 'min', 'max']
        ,'MONTHS_BALANCE': ['mean', 'std', 'min', 'max']
        ,'AMT_BALANCE': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_LIMIT_ACTUAL': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_ATM_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_OTHER_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_POS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_INST_MIN_REGULARITY': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT_TOTAL_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_RECEIVABLE_PRINCIPAL': ['mean', 'std', 'min', 'max']
        ,'AMT_RECIVABLE': ['mean', 'std', 'min', 'max']
        ,'AMT_TOTAL_RECEIVABLE': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_ATM_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_OTHER_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_POS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_INSTALMENT_MATURE_CUM': ['mean', 'std', 'min', 'max']
        ,'SK_DPD': ['mean', 'std', 'min', 'max']
        ,'SK_DPD_DEF': ['mean', 'std', 'min', 'max']
        # 
        ,'NAME_CONTRACT_STATUS_Active': ['mean']
        ,'NAME_CONTRACT_STATUS_Approved': ['mean']
        ,'NAME_CONTRACT_STATUS_Completed': ['mean']
        ,'NAME_CONTRACT_STATUS_Demand': ['mean']
        ,'NAME_CONTRACT_STATUS_Refused': ['mean']
        ,'NAME_CONTRACT_STATUS_Sent proposal': ['mean']
        ,'NAME_CONTRACT_STATUS_Signed': ['mean']
        ,'NAME_CONTRACT_STATUS_nan': ['mean']
        # IDのユニーク数をカウント
        ,"SK_ID_PREV":["count", "nunique"]
    }
)

# カラム名の付与
credit_ohe_agg.columns = [i + "_" + j for i,j in credit_ohe_agg.columns]
credit_ohe_agg = credit_ohe_agg.reset_index(drop=False)

print(credit_ohe_agg.shape)
credit_ohe_agg.head()

(103558, 95)


,SK_ID_CURR,SK_ID_CURR_mean,SK_ID_CURR_std,SK_ID_CURR_min,SK_ID_CURR_max,MONTHS_BALANCE_mean,MONTHS_BALANCE_std,MONTHS_BALANCE_min,MONTHS_BALANCE_max,AMT_BALANCE_mean,...,NAME_CONTRACT_STATUS_Active_mean,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Refused_mean,NAME_CONTRACT_STATUS_Sent proposal_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100006,100006.0,0.0,100006,100006,-3.5,1.870829,-6,-1,0.000000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,6,1
1,100011,100011.0,0.0,100011,100011,-38.5,21.505813,-75,-2,54482.109375,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,74,1
2,100013,100013.0,0.0,100013,100013,-48.5,27.856777,-96,-1,18159.919922,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,96,1
3,100021,100021.0,0.0,100021,100021,-10.0,5.049752,-18,-2,0.000000,...,0.411765,0.0,0.588235,0.0,0.0,0.0,0.0,0.0,17,1
4,100023,100023.0,0.0,100023,100023,-7.5,2.449490,-11,-4,0.000000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,8,1


In [17]:
previous = previous_application_origin.copy()
previous = baseline.reduce_mem_usage(credit)
print(previous.shape)
previous.head()

Memory usage of dataframe is 318.63 MB
Memory usage of optimization is 334107272.00 MB
Decreased by -104857500.0%
(3840312, 23)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970001,135000,0.0,877.5,0.0,877.5,1700.324951,...,0.000000,0.000000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.554688,45000,2250.0,2250.0,0.0,0.0,2250.000000,...,64875.554688,64875.554688,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.224609,450000,0.0,0.0,0.0,0.0,2250.000000,...,31460.085938,31460.085938,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.109375,225000,2250.0,2250.0,0.0,0.0,11795.759766,...,233048.968750,233048.968750,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.468750,450000,0.0,11547.0,0.0,11547.0,22924.890625,...,453919.468750,453919.468750,0.0,1,0.0,1.0,101.0,Active,0,0


In [18]:
# one-hotencoding(処理対象カラムを取得)
encoding_columns = []

for col in previous.columns:
    if previous[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

#encoding_columns

NAME_CONTRACT_STATUS


In [19]:
# one-hot-encoding
for col in encoding_columns:
    previous_ohe = pd.get_dummies(previous, columns=[col], dummy_na=True)
    col_ohe = sorted(list(set(previous_ohe.columns) - set(previous.columns)))

print(len(col_ohe))
col_ohe

8


['NAME_CONTRACT_STATUS_Active',
 'NAME_CONTRACT_STATUS_Approved',
 'NAME_CONTRACT_STATUS_Completed',
 'NAME_CONTRACT_STATUS_Demand',
 'NAME_CONTRACT_STATUS_Refused',
 'NAME_CONTRACT_STATUS_Sent proposal',
 'NAME_CONTRACT_STATUS_Signed',
 'NAME_CONTRACT_STATUS_nan']

In [20]:
previous_ohe_agg = credit_ohe.groupby('SK_ID_CURR').agg(
    {
        # 数値変数の集約
        'MONTHS_BALANCE': ['mean', 'std', 'min', 'max']
        ,'AMT_BALANCE': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_LIMIT_ACTUAL': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_ATM_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_OTHER_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_POS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_INST_MIN_REGULARITY': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT_TOTAL_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_RECEIVABLE_PRINCIPAL': ['mean', 'std', 'min', 'max']
        ,'AMT_RECIVABLE': ['mean', 'std', 'min', 'max']
        ,'AMT_TOTAL_RECEIVABLE': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_ATM_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_OTHER_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_POS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_INSTALMENT_MATURE_CUM': ['mean', 'std', 'min', 'max']
        ,'SK_DPD': ['mean', 'std', 'min', 'max']
        ,'SK_DPD_DEF': ['mean', 'std', 'min', 'max']
        # 
        ,'NAME_CONTRACT_STATUS_Active': ['mean']
        ,'NAME_CONTRACT_STATUS_Approved': ['mean']
        ,'NAME_CONTRACT_STATUS_Completed': ['mean']
        ,'NAME_CONTRACT_STATUS_Demand': ['mean']
        ,'NAME_CONTRACT_STATUS_Refused': ['mean']
        ,'NAME_CONTRACT_STATUS_Sent proposal': ['mean']
        ,'NAME_CONTRACT_STATUS_Signed': ['mean']
        ,'NAME_CONTRACT_STATUS_nan': ['mean']
        # IDのユニーク数をカウント
        ,"SK_ID_PREV":["count", "nunique"]
    }
)

# カラム名の付与
previous_ohe_agg.columns = [i + "_" + j for i,j in previous_ohe_agg.columns]
previous_ohe_agg = previous_ohe_agg.reset_index(drop=False)

print(previous_ohe_agg.shape)
previous_ohe_agg.head()

(103558, 91)


,SK_ID_CURR,MONTHS_BALANCE_mean,MONTHS_BALANCE_std,MONTHS_BALANCE_min,MONTHS_BALANCE_max,AMT_BALANCE_mean,AMT_BALANCE_std,AMT_BALANCE_min,AMT_BALANCE_max,AMT_CREDIT_LIMIT_ACTUAL_mean,...,NAME_CONTRACT_STATUS_Active_mean,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Refused_mean,NAME_CONTRACT_STATUS_Sent proposal_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100006,-3.5,1.870829,-6,-1,0.000000,0.000000,0.0,0.00000,270000.000000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,6,1
1,100011,-38.5,21.505813,-75,-2,54482.109375,68127.238133,0.0,189000.00000,164189.189189,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,74,1
2,100013,-48.5,27.856777,-96,-1,18159.919922,43237.406465,0.0,161420.21875,131718.750000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,96,1
3,100021,-10.0,5.049752,-18,-2,0.000000,0.000000,0.0,0.00000,675000.000000,...,0.411765,0.0,0.588235,0.0,0.0,0.0,0.0,0.0,17,1
4,100023,-7.5,2.449490,-11,-4,0.000000,0.000000,0.0,0.00000,135000.000000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,8,1


In [21]:
installments = installments_payments_origin.copy()
installments = baseline.reduce_mem_usage(installments)
print(installments.shape)
installments.head()

Memory usage of dataframe is 830.41 MB
Memory usage of optimization is 408162158.00 MB
Decreased by -49151908.2%
(13605401, 8)


,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.359863,6948.359863
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525024,1716.525024
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000000,25425.000000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130859,24350.130859
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040039,2160.584961


In [22]:
# one-hotencoding(処理対象カラムを取得)
encoding_columns = []

for col in installments.columns:
    if installments[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

encoding_columns

[]

In [23]:
# one-hot-encoding
#for col in encoding_columns:
#    installments_ohe = pd.get_dummies(installments, columns=[col], dummy_na=True)
#    col_ohe = sorted(list(set(installments_ohe.columns) - set(installments.columns)))

#print(len(col_ohe))
#col_ohe
installments_ohe = installments.copy()

In [24]:
installments_ohe = installments_ohe.rename(columns={'SK_ID_PREV': 'SK_ID_PREV_IP'})

In [25]:
installments_ohe_agg = installments_ohe.groupby('SK_ID_CURR').agg(
    {
        # 数値変数の集約
        'NUM_INSTALMENT_VERSION': ['mean', 'std', 'min', 'max']
        ,'NUM_INSTALMENT_NUMBER': ['mean', 'std', 'min', 'max']
        ,'DAYS_INSTALMENT': ['mean', 'std', 'min', 'max']
        ,'DAYS_ENTRY_PAYMENT': ['mean', 'std', 'min', 'max']
        ,'AMT_INSTALMENT': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT': ['mean', 'std', 'min', 'max']
        # IDのユニーク数をカウント
        ,"SK_ID_PREV_IP":["count", "nunique"]
    }
)

# カラム名の付与
installments_ohe_agg.columns = [i + "_" + j for i,j in installments_ohe_agg.columns]
installments_ohe_agg = installments_ohe_agg.reset_index(drop=False)

print(installments_ohe_agg.shape)
installments_ohe_agg.head()

(339587, 27)


,SK_ID_CURR,NUM_INSTALMENT_VERSION_mean,NUM_INSTALMENT_VERSION_std,NUM_INSTALMENT_VERSION_min,NUM_INSTALMENT_VERSION_max,NUM_INSTALMENT_NUMBER_mean,NUM_INSTALMENT_NUMBER_std,NUM_INSTALMENT_NUMBER_min,NUM_INSTALMENT_NUMBER_max,DAYS_INSTALMENT_mean,...,AMT_INSTALMENT_mean,AMT_INSTALMENT_std,AMT_INSTALMENT_min,AMT_INSTALMENT_max,AMT_PAYMENT_mean,AMT_PAYMENT_std,AMT_PAYMENT_min,AMT_PAYMENT_max,SK_ID_PREV_IP_count,SK_ID_PREV_IP_nunique
0,100001,1.142857,0.377964,1.0,2.0,2.714286,1.112697,1,4,-2187.714355,...,5885.132324,5076.676763,3951.000000,17397.900391,5885.132324,5076.676763,3951.000000,17397.900391,7,2
1,100002,1.052632,0.229416,1.0,2.0,10.000000,5.627314,1,19,-295.000000,...,11559.247070,10058.037883,9251.775391,53093.746094,11559.247070,10058.037883,9251.775391,53093.746094,19,1
2,100003,1.040000,0.200000,1.0,2.0,5.080000,3.134751,1,12,-1378.160034,...,64754.585938,110542.594872,6662.970215,560835.375000,64754.585938,110542.594872,6662.970215,560835.375000,25,3
3,100004,1.333333,0.577350,1.0,2.0,2.000000,1.000000,1,3,-754.000000,...,7096.154785,3011.871719,5357.250000,10573.964844,7096.154785,3011.871719,5357.250000,10573.964844,3,1
4,100005,1.111111,0.333333,1.0,2.0,5.000000,2.738613,1,9,-586.000000,...,6240.205078,4281.014648,4813.200195,17656.244141,6240.205078,4281.014648,4813.200195,17656.244141,9,1


In [26]:
bureau = bureau_origin.copy()
bureau = baseline.reduce_mem_usage(bureau)
print(bureau.shape)
bureau.head()

Memory usage of dataframe is 222.62 MB
Memory usage of optimization is 125299372.00 MB
Decreased by -56283785.5%
(1716428, 17)


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [27]:
### 
for col in bureau.columns:
    bureau[col] = bureau[col].replace('\s', '_', regex=True)
    bureau[col] = bureau[col].replace(':', '_', regex=True)
    bureau[col] = bureau[col].replace('/', '_', regex=True)
    
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency_1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer_credit,-131,NaN
1,215354,5714463,Active,currency_1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit_card,-20,NaN
2,215354,5714464,Active,currency_1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer_credit,-16,NaN
3,215354,5714465,Active,currency_1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit_card,-16,NaN
4,215354,5714466,Active,currency_1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer_credit,-21,NaN


In [28]:
# one-hotencoding(処理対象カラムを取得)
encoding_columns = []

for col in bureau.columns:
    if bureau[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

CREDIT_ACTIVE
CREDIT_CURRENCY
CREDIT_TYPE


In [29]:
# one-hot-encoding
bureau_ohe = pd.get_dummies(bureau, columns=['CREDIT_ACTIVE'], dummy_na=True)
col_ohe = sorted(list(set(bureau_ohe.columns) - set(bureau.columns)))

bureau_ohe = pd.get_dummies(bureau_ohe, columns=['CREDIT_CURRENCY'], dummy_na=True)
col_ohe = sorted(list(set(bureau_ohe.columns) - set(bureau.columns)))

bureau_ohe = pd.get_dummies(bureau_ohe, columns=['CREDIT_TYPE'], dummy_na=True)
col_ohe = sorted(list(set(bureau_ohe.columns) - set(bureau.columns)))

print(len(col_ohe))
col_ohe

26


['CREDIT_ACTIVE_Active',
 'CREDIT_ACTIVE_Bad_debt',
 'CREDIT_ACTIVE_Closed',
 'CREDIT_ACTIVE_Sold',
 'CREDIT_ACTIVE_nan',
 'CREDIT_CURRENCY_currency_1',
 'CREDIT_CURRENCY_currency_2',
 'CREDIT_CURRENCY_currency_3',
 'CREDIT_CURRENCY_currency_4',
 'CREDIT_CURRENCY_nan',
 'CREDIT_TYPE_Another_type_of_loan',
 'CREDIT_TYPE_Car_loan',
 'CREDIT_TYPE_Cash_loan_(non-earmarked)',
 'CREDIT_TYPE_Consumer_credit',
 'CREDIT_TYPE_Credit_card',
 'CREDIT_TYPE_Interbank_credit',
 'CREDIT_TYPE_Loan_for_business_development',
 'CREDIT_TYPE_Loan_for_purchase_of_shares_(margin_lending)',
 'CREDIT_TYPE_Loan_for_the_purchase_of_equipment',
 'CREDIT_TYPE_Loan_for_working_capital_replenishment',
 'CREDIT_TYPE_Microloan',
 'CREDIT_TYPE_Mobile_operator_loan',
 'CREDIT_TYPE_Mortgage',
 'CREDIT_TYPE_Real_estate_loan',
 'CREDIT_TYPE_Unknown_type_of_loan',
 'CREDIT_TYPE_nan']

In [30]:
bureau_ohe = bureau_ohe.rename(columns={'SK_ID_BUREAU': 'SK_ID_BUREAU_BUREAU'})
bureau_ohe.head()

,SK_ID_CURR,SK_ID_BUREAU_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,...,CREDIT_TYPE_Loan_for_business_development,CREDIT_TYPE_Loan_for_purchase_of_shares_(margin_lending),CREDIT_TYPE_Loan_for_the_purchase_of_equipment,CREDIT_TYPE_Loan_for_working_capital_replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile_operator_loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real_estate_loan,CREDIT_TYPE_Unknown_type_of_loan,CREDIT_TYPE_nan
0,215354,5714462,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,215354,5714463,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,...,0,0,0,0,0,0,0,0,0,0
2,215354,5714464,-203,0,528.0,NaN,NaN,0,464323.5,NaN,...,0,0,0,0,0,0,0,0,0,0
3,215354,5714465,-203,0,NaN,NaN,NaN,0,90000.0,NaN,...,0,0,0,0,0,0,0,0,0,0
4,215354,5714466,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,...,0,0,0,0,0,0,0,0,0,0


In [31]:
bureau_ohe_agg = bureau_ohe.groupby('SK_ID_CURR').agg(
    {
        # 数値変数の集約
        'DAYS_CREDIT': ['mean', 'std', 'min', 'max', 'var']
        ,'CREDIT_DAY_OVERDUE': ['mean', 'std', 'min', 'max']
        ,'DAYS_CREDIT_ENDDATE': ['mean', 'std', 'min', 'max']
        ,'DAYS_ENDDATE_FACT': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_MAX_OVERDUE': ['mean', 'std', 'min', 'max']
        ,'CNT_CREDIT_PROLONG': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_SUM': ['mean', 'std', 'min', 'max', 'sum']
        ,'AMT_CREDIT_SUM_DEBT': ['mean', 'std', 'min', 'max', 'sum']
        ,'AMT_CREDIT_SUM_LIMIT': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_SUM_OVERDUE': ['mean', 'std', 'min', 'max']
        ,'DAYS_CREDIT_UPDATE': ['mean', 'std', 'min', 'max']
        ,'AMT_ANNUITY': ['mean', 'std', 'min', 'max']
        # 
        ,'CREDIT_TYPE_Another_type_of_loan': ['mean']
        ,'CREDIT_TYPE_Car_loan': ['mean']
        ,'CREDIT_TYPE_Cash_loan_(non-earmarked)': ['mean']
        ,'CREDIT_TYPE_Consumer_credit': ['mean']
        ,'CREDIT_TYPE_Credit_card': ['mean']
        ,'CREDIT_TYPE_Interbank_credit': ['mean']
        ,'CREDIT_TYPE_Loan_for_business_development': ['mean']
        ,'CREDIT_TYPE_Loan_for_purchase_of_shares_(margin_lending)': ['mean']
        ,'CREDIT_TYPE_Loan_for_the_purchase_of_equipment': ['mean']
        ,'CREDIT_TYPE_Loan_for_working_capital_replenishment': ['mean']
        ,'CREDIT_TYPE_Microloan': ['mean']
        ,'CREDIT_TYPE_Mobile_operator_loan': ['mean']
        ,'CREDIT_TYPE_Mortgage': ['mean']
        ,'CREDIT_TYPE_Real_estate_loan': ['mean']
        ,'CREDIT_TYPE_Unknown_type_of_loan': ['mean']
        ,'CREDIT_TYPE_nan': ['mean']
        ,'CREDIT_ACTIVE_Active': ['mean']
        ,'CREDIT_ACTIVE_Bad_debt': ['mean']
        ,'CREDIT_ACTIVE_Closed': ['mean']
        ,'CREDIT_ACTIVE_Sold': ['mean']
        ,'CREDIT_ACTIVE_nan': ['mean']
        ,'CREDIT_CURRENCY_currency_1': ['mean']
        ,'CREDIT_CURRENCY_currency_2': ['mean']
        ,'CREDIT_CURRENCY_currency_3': ['mean']
        ,'CREDIT_CURRENCY_currency_4': ['mean']
        ,'CREDIT_CURRENCY_nan': ['mean']
        ,'CREDIT_TYPE_Another_type_of_loan': ['mean']
        ,'CREDIT_TYPE_Car_loan': ['mean']
        ,'CREDIT_TYPE_Cash_loan_(non-earmarked)': ['mean']
        ,'CREDIT_TYPE_Consumer_credit': ['mean']
        ,'CREDIT_TYPE_Credit_card': ['mean']
        ,'CREDIT_TYPE_Interbank_credit': ['mean']
        ,'CREDIT_TYPE_Loan_for_business_development': ['mean']
        ,'CREDIT_TYPE_Loan_for_purchase_of_shares_(margin_lending)': ['mean']
        ,'CREDIT_TYPE_Loan_for_the_purchase_of_equipment': ['mean']
        ,'CREDIT_TYPE_Loan_for_working_capital_replenishment': ['mean']
        ,'CREDIT_TYPE_Microloan': ['mean']
        ,'CREDIT_TYPE_Mobile_operator_loan': ['mean']
        ,'CREDIT_TYPE_Mortgage': ['mean']
        ,'CREDIT_TYPE_Real_estate_loan': ['mean']
        ,'CREDIT_TYPE_Unknown_type_of_loan': ['mean']
        ,'CREDIT_TYPE_nan': ['mean']
        # IDのユニーク数をカウント
        ,'SK_ID_BUREAU_BUREAU': ["count", "nunique"]
    }
)

# カラム名の付与
bureau_ohe_agg.columns = [i + "_" + j for i,j in bureau_ohe_agg.columns]
bureau_ohe_agg = bureau_ohe_agg.reset_index(drop=False)

#
bureau_ohe_agg['debt_credit_ratio_None'] = bureau_ohe_agg['AMT_CREDIT_SUM_DEBT_sum'] / bureau_ohe_agg['AMT_CREDIT_SUM_sum']

print(bureau_ohe_agg.shape)
bureau_ohe_agg.head()

(305811, 81)


,SK_ID_CURR,DAYS_CREDIT_mean,DAYS_CREDIT_std,DAYS_CREDIT_min,DAYS_CREDIT_max,DAYS_CREDIT_var,CREDIT_DAY_OVERDUE_mean,CREDIT_DAY_OVERDUE_std,CREDIT_DAY_OVERDUE_min,CREDIT_DAY_OVERDUE_max,...,CREDIT_ACTIVE_Sold_mean,CREDIT_ACTIVE_nan_mean,CREDIT_CURRENCY_currency_1_mean,CREDIT_CURRENCY_currency_2_mean,CREDIT_CURRENCY_currency_3_mean,CREDIT_CURRENCY_currency_4_mean,CREDIT_CURRENCY_nan_mean,SK_ID_BUREAU_BUREAU_count,SK_ID_BUREAU_BUREAU_nunique,debt_credit_ratio_None
0,100001,-735.000000,489.942514,-1572,-49,240043.666667,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7,7,0.410555
1,100002,-874.000000,431.451040,-1437,-103,186150.000000,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8,8,0.284122
2,100003,-1400.750000,909.826128,-2586,-606,827783.583333,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4,4,0.000000
3,100004,-867.000000,649.124025,-1326,-408,421362.000000,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2,2,0.000000
4,100005,-190.666667,162.297053,-373,-62,26340.333333,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3,3,0.864992


In [32]:
### bureau_balance
df = df.merge(bureau_ohe_agg, how='left', on='SK_ID_CURR')
df.head()

,index,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,CREDIT_ACTIVE_Sold_mean,CREDIT_ACTIVE_nan_mean,CREDIT_CURRENCY_currency_1_mean,CREDIT_CURRENCY_currency_2_mean,CREDIT_CURRENCY_currency_3_mean,CREDIT_CURRENCY_currency_4_mean,CREDIT_CURRENCY_nan_mean,SK_ID_BUREAU_BUREAU_count,SK_ID_BUREAU_BUREAU_nunique,debt_credit_ratio_None
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8.0,8.0,0.284122
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,4.0,0.000000
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,0.000000
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.000000


In [33]:
### previos_application
df = df.merge(previous_ohe_agg, how='left', on='SK_ID_CURR')
df.head()

,index,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Active_mean,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Refused_mean,NAME_CONTRACT_STATUS_Sent proposal_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,1.0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
### pos_cash
df = df.merge(pos_ohe_agg, how='left', on='SK_ID_CURR')
df.head()

,index,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Approved_mean_y,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean_y,NAME_CONTRACT_STATUS_Demand_mean_y,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean_y,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean_y,SK_ID_PREV_count_y,SK_ID_PREV_nunique_y
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,19.0,1.0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,...,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.0,0.0,28.0,3.0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,...,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0,0.0,4.0,1.0
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,...,0.0,0.0,0.095238,0.0,0.047619,0.000000,0.0,0.0,21.0,3.0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,...,0.0,0.0,0.045455,0.0,0.000000,0.015152,0.0,0.0,66.0,5.0


In [35]:
### instllment_payment
df = df.merge(installments_ohe_agg, how='left', on='SK_ID_CURR')
df.head()

,index,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,AMT_INSTALMENT_mean,AMT_INSTALMENT_std,AMT_INSTALMENT_min,AMT_INSTALMENT_max,AMT_PAYMENT_mean,AMT_PAYMENT_std,AMT_PAYMENT_min,AMT_PAYMENT_max,SK_ID_PREV_IP_count,SK_ID_PREV_IP_nunique
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,...,11559.247070,10058.037883,9251.775391,53093.746094,11559.247070,10058.037883,9251.775391,53093.746094,19.0,1.0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,...,64754.585938,110542.594872,6662.970215,560835.375000,64754.585938,110542.594872,6662.970215,560835.375000,25.0,3.0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,...,7096.154785,3011.871719,5357.250000,10573.964844,7096.154785,3011.871719,5357.250000,10573.964844,3.0,1.0
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,...,62947.085938,168097.620686,2482.919922,691786.875000,62947.085938,168097.620686,2482.919922,691786.875000,16.0,3.0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,...,12666.444336,7852.910602,1821.780029,22678.785156,12214.060547,8048.060783,0.180000,22678.785156,66.0,5.0


In [36]:
### credit_card_balance
df = df.merge(credit_ohe_agg, how='left', on='SK_ID_CURR')
df.head()

,index,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Active_mean,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Refused_mean_y,NAME_CONTRACT_STATUS_Sent proposal_mean_y,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,1.0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
### infをnanへ置換
df = df.replace([np.inf, -np.inf], np.nan)

In [38]:
### 変数選択
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 569 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 552 features are remained after removing non-informative features
80 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 472 features are remained after removing features not interesting for LightGBM classifier
---=> 48 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 520 features

---=> df final shape: (356250, 520)  <=--- 



,index,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,MONTHS_BALANCE_max_x_high_risk,MONTHS_BALANCE_max_x_low_risk,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NAME_CONTRACT_STATUS_Canceled_mean_high_risk,NAME_CONTRACT_STATUS_Canceled_mean_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,NUM_INSTALMENT_VERSION_min_low_risk,SK_ID_PREV_nunique_high_risk,SK_ID_PREV_nunique_low_risk
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,...,0,0,0,0,1,0,0,1,0,0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,...,0,0,0,0,1,0,0,1,0,0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,...,0,0,0,0,1,0,0,1,0,0
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,...,1,0,1,0,1,0,0,1,1,0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,...,0,0,0,0,1,0,0,1,0,0


In [39]:
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df.head()

,index,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,MONTHS_BALANCE_max_x_high_risk,MONTHS_BALANCE_max_x_low_risk,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NAME_CONTRACT_STATUS_Canceled_mean_high_risk,NAME_CONTRACT_STATUS_Canceled_mean_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,NUM_INSTALMENT_VERSION_min_low_risk,SK_ID_PREV_nunique_high_risk,SK_ID_PREV_nunique_low_risk
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,...,0,0,0,0,1,0,0,1,0,0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,...,0,0,0,0,1,0,0,1,0,0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,...,0,0,0,0,1,0,0,1,0,0
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,...,1,0,1,0,1,0,0,1,1,0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,...,0,0,0,0,1,0,0,1,0,0


In [40]:
# split train, and test datasets
df_train = df[df['TARGET'].notnull()]
df_test = df[df['TARGET'].isnull()]

print(df_test.shape)
df_test.head()

(48744, 520)


,index,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,MONTHS_BALANCE_max_x_high_risk,MONTHS_BALANCE_max_x_low_risk,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NAME_CONTRACT_STATUS_Canceled_mean_high_risk,NAME_CONTRACT_STATUS_Canceled_mean_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,NUM_INSTALMENT_VERSION_min_low_risk,SK_ID_PREV_nunique_high_risk,SK_ID_PREV_nunique_low_risk
307506,0,100001,NaN,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,...,0,0,0,0,1,0,0,1,0,0
307507,1,100005,NaN,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,...,0,0,0,0,1,0,0,1,0,0
307508,2,100013,NaN,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,...,1,0,1,0,1,0,1,0,1,0
307509,3,100028,NaN,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,...,1,0,1,0,1,0,1,0,1,0
307510,4,100038,NaN,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,...,0,0,0,0,1,0,0,1,0,0


In [41]:
pred_train_1b = pd.read_csv('./home-credit-default-risk/exports/pred_train_1b.csv')
pred_test_1b =  pd.read_csv('./home-credit-default-risk/exports/pred_test_1b.csv')
pred_test_1b.head()

,SK_ID_CURR,pred_1b
0,100001,0.051055
1,100005,0.166067
2,100013,0.043899
3,100028,0.056542
4,100038,0.183168


In [42]:
df_train = df_train.merge(pred_train_1b, how='left', on='SK_ID_CURR')
df_test = df_test.merge(pred_test_1b, how='left', on='SK_ID_CURR')
df_test.head()

,index,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,MONTHS_BALANCE_max_x_low_risk,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NAME_CONTRACT_STATUS_Canceled_mean_high_risk,NAME_CONTRACT_STATUS_Canceled_mean_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,NUM_INSTALMENT_VERSION_min_low_risk,SK_ID_PREV_nunique_high_risk,SK_ID_PREV_nunique_low_risk,pred_1b
0,0,100001,NaN,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,...,0,0,0,1,0,0,1,0,0,0.051055
1,1,100005,NaN,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,...,0,0,0,1,0,0,1,0,0,0.166067
2,2,100013,NaN,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,...,0,1,0,1,0,1,0,1,0,0.043899
3,3,100028,NaN,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,...,0,1,0,1,0,1,0,1,0,0.056542
4,4,100038,NaN,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,...,0,0,0,1,0,0,1,0,0,0.183168


In [43]:
pred_train_1a = pd.read_csv('./home-credit-default-risk/exports/pred_train_1a.csv')
pred_test_1a =  pd.read_csv('./home-credit-default-risk/exports/pred_test_1a.csv')
pred_test_1b.head()

,SK_ID_CURR,pred_1b
0,100001,0.051055
1,100005,0.166067
2,100013,0.043899
3,100028,0.056542
4,100038,0.183168


In [44]:
df_train = df_train.merge(pred_train_1a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(pred_test_1a, how='left', on='SK_ID_CURR')
df_test.head()

,index,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NAME_CONTRACT_STATUS_Canceled_mean_high_risk,NAME_CONTRACT_STATUS_Canceled_mean_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,NUM_INSTALMENT_VERSION_min_low_risk,SK_ID_PREV_nunique_high_risk,SK_ID_PREV_nunique_low_risk,pred_1b,pred_1a
0,0,100001,NaN,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,...,0,0,1,0,0,1,0,0,0.051055,0.038227
1,1,100005,NaN,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,...,0,0,1,0,0,1,0,0,0.166067,0.112768
2,2,100013,NaN,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,...,1,0,1,0,1,0,1,0,0.043899,0.033638
3,3,100028,NaN,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,...,1,0,1,0,1,0,1,0,0.056542,0.031213
4,4,100038,NaN,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,...,0,0,1,0,0,1,0,0,0.183168,0.181109


In [45]:
pred_train_set1a = pd.read_csv('./home-credit-default-risk/exports/pred_train_set1a.csv')
pred_test_set1a =  pd.read_csv('./home-credit-default-risk/exports/pred_test_set1a.csv')
pred_test_set1a.head()

,SK_ID_CURR,set1
0,100001,0.029195
1,100005,0.101040
2,100013,0.031550
3,100028,0.040153
4,100038,0.115963


In [46]:
df_train = df_train.merge(pred_train_set1a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(pred_test_set1a, how='left', on='SK_ID_CURR')
df_test.head()

,index,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,SK_ID_PREV_nunique_x_low_risk,NAME_CONTRACT_STATUS_Canceled_mean_high_risk,NAME_CONTRACT_STATUS_Canceled_mean_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,NUM_INSTALMENT_VERSION_min_low_risk,SK_ID_PREV_nunique_high_risk,SK_ID_PREV_nunique_low_risk,pred_1b,pred_1a,set1
0,0,100001,NaN,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,...,0,1,0,0,1,0,0,0.051055,0.038227,0.029195
1,1,100005,NaN,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,...,0,1,0,0,1,0,0,0.166067,0.112768,0.101040
2,2,100013,NaN,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,...,0,1,0,1,0,1,0,0.043899,0.033638,0.031550
3,3,100028,NaN,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,...,0,1,0,1,0,1,0,0.056542,0.031213,0.040153
4,4,100038,NaN,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,...,0,1,0,0,1,0,0,0.183168,0.181109,0.115963


In [47]:
train_set2a = pd.read_csv('./home-credit-default-risk/exports/train_set2a.csv')
test_set2a =  pd.read_csv('./home-credit-default-risk/exports/test_set2a.csv')
test_set2a.head()

,SK_ID_CURR,set2a
0,100001,0.025023
1,100005,0.090335
2,100013,0.037033
3,100028,0.036644
4,100038,0.172674


In [48]:
df_train = df_train.merge(train_set2a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set2a, how='left', on='SK_ID_CURR')
df_test.head()

,index,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Canceled_mean_high_risk,NAME_CONTRACT_STATUS_Canceled_mean_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,NUM_INSTALMENT_VERSION_min_low_risk,SK_ID_PREV_nunique_high_risk,SK_ID_PREV_nunique_low_risk,pred_1b,pred_1a,set1,set2a
0,0,100001,NaN,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,...,1,0,0,1,0,0,0.051055,0.038227,0.029195,0.025023
1,1,100005,NaN,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,...,1,0,0,1,0,0,0.166067,0.112768,0.101040,0.090335
2,2,100013,NaN,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,...,1,0,1,0,1,0,0.043899,0.033638,0.031550,0.037033
3,3,100028,NaN,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,...,1,0,1,0,1,0,0.056542,0.031213,0.040153,0.036644
4,4,100038,NaN,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,...,1,0,0,1,0,0,0.183168,0.181109,0.115963,0.172674


In [49]:
### infをnanへ置換
df_train = df_train.replace([np.inf, -np.inf], np.nan)
df_train.head()

,index,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Canceled_mean_high_risk,NAME_CONTRACT_STATUS_Canceled_mean_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,NUM_INSTALMENT_VERSION_min_low_risk,SK_ID_PREV_nunique_high_risk,SK_ID_PREV_nunique_low_risk,pred_1b,pred_1a,set1,set2a
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,...,1,0,0,1,0,0,0.516856,0.162368,0.177023,0.156048
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,...,1,0,0,1,0,0,0.010645,0.027249,0.012354,0.012034
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,...,1,0,0,1,0,0,0.054077,0.030869,0.040684,0.026333
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,...,1,0,0,1,1,0,0.059492,0.029480,0.030685,0.029856
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,...,1,0,0,1,0,0,0.076165,0.047569,0.041810,0.034698


In [50]:
### infをnanへ置換
df_test = df_test.replace([np.inf, -np.inf], np.nan)
df_test.head()

,index,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Canceled_mean_high_risk,NAME_CONTRACT_STATUS_Canceled_mean_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,NUM_INSTALMENT_VERSION_min_low_risk,SK_ID_PREV_nunique_high_risk,SK_ID_PREV_nunique_low_risk,pred_1b,pred_1a,set1,set2a
0,0,100001,NaN,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,...,1,0,0,1,0,0,0.051055,0.038227,0.029195,0.025023
1,1,100005,NaN,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,...,1,0,0,1,0,0,0.166067,0.112768,0.101040,0.090335
2,2,100013,NaN,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,...,1,0,1,0,1,0,0.043899,0.033638,0.031550,0.037033
3,3,100028,NaN,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,...,1,0,1,0,1,0,0.056542,0.031213,0.040153,0.036644
4,4,100038,NaN,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,...,1,0,0,1,0,0,0.183168,0.181109,0.115963,0.172674


In [51]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)
encoding_columns.remove('SK_ID_CURR')
encoding_columns.remove('TARGET')

In [52]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

,index,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Canceled_mean_high_risk,NAME_CONTRACT_STATUS_Canceled_mean_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,NUM_INSTALMENT_VERSION_min_low_risk,SK_ID_PREV_nunique_high_risk,SK_ID_PREV_nunique_low_risk,pred_1b,pred_1a,set1,set2a
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,0,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,...,1,0,0,1,0,0,0.516856,0.162368,0.177023,0.156048
100003,1,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,...,1,0,0,1,0,0,0.010645,0.027249,0.012354,0.012034
100004,2,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,...,1,0,0,1,0,0,0.054077,0.030869,0.040684,0.026333
100006,3,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,...,1,0,0,1,1,0,0.059492,0.029480,0.030685,0.029856
100007,4,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,...,1,0,0,1,0,0,0.076165,0.047569,0.041810,0.034698


In [53]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

,index,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Canceled_mean_high_risk,NAME_CONTRACT_STATUS_Canceled_mean_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,NUM_INSTALMENT_VERSION_min_low_risk,SK_ID_PREV_nunique_high_risk,SK_ID_PREV_nunique_low_risk,pred_1b,pred_1a,set1,set2a
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,-1.732065,1.0,-0.577531,0.313221,-0.478099,-0.166152,-0.507239,-0.149780,1.506891,0.747070,...,0.249915,0.0,-0.502076,0.581919,-0.627631,0.0,5.053162,0.986709,1.099371,0.860543
100003,-1.732054,0.0,-0.577531,0.933556,1.725433,0.592656,1.599963,-1.252930,-0.166811,0.511719,...,0.249915,0.0,-0.502076,0.581919,-0.627631,0.0,-0.839862,-0.548398,-0.654951,-0.659948
100004,-1.732043,0.0,-0.577531,-0.927449,-1.152888,-1.404647,-1.091896,-0.783203,-0.689500,0.923828,...,0.249915,0.0,-0.502076,0.581919,-0.627631,0.0,-0.334248,-0.507271,-0.353138,-0.508977
100006,-1.732031,0.0,-0.577531,-0.307114,-0.711432,0.177857,-0.653403,-0.928711,-0.680105,-0.280518,...,0.249915,0.0,-0.502076,0.581919,1.593292,0.0,-0.271209,-0.523054,-0.459667,-0.471789
100007,-1.732020,0.0,-0.577531,-0.431181,-0.213740,-0.361753,-0.068746,0.562988,-0.892525,-0.279785,...,0.249915,0.0,-0.502076,0.581919,-0.627631,0.0,-0.077109,-0.317542,-0.341142,-0.420660


In [54]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

,SK_ID_CURR,index,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Canceled_mean_high_risk,NAME_CONTRACT_STATUS_Canceled_mean_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,NUM_INSTALMENT_VERSION_min_low_risk,SK_ID_PREV_nunique_high_risk,SK_ID_PREV_nunique_low_risk,pred_1b,pred_1a,set1,set2a
0,100002,-1.732065,1.0,-0.577531,0.313221,-0.478099,-0.166152,-0.507239,-0.149780,1.506891,...,0.249915,0.0,-0.502076,0.581919,-0.627631,0.0,5.053162,0.986709,1.099371,0.860543
1,100003,-1.732054,0.0,-0.577531,0.933556,1.725433,0.592656,1.599963,-1.252930,-0.166811,...,0.249915,0.0,-0.502076,0.581919,-0.627631,0.0,-0.839862,-0.548398,-0.654951,-0.659948
2,100004,-1.732043,0.0,-0.577531,-0.927449,-1.152888,-1.404647,-1.091896,-0.783203,-0.689500,...,0.249915,0.0,-0.502076,0.581919,-0.627631,0.0,-0.334248,-0.507271,-0.353138,-0.508977
3,100006,-1.732031,0.0,-0.577531,-0.307114,-0.711432,0.177857,-0.653403,-0.928711,-0.680105,...,0.249915,0.0,-0.502076,0.581919,1.593292,0.0,-0.271209,-0.523054,-0.459667,-0.471789
4,100007,-1.732020,0.0,-0.577531,-0.431181,-0.213740,-0.361753,-0.068746,0.562988,-0.892525,...,0.249915,0.0,-0.502076,0.581919,-0.627631,0.0,-0.077109,-0.317542,-0.341142,-0.420660


In [55]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 632.27 MB
Memory usage of optimization is 644532704.00 MB
Decreased by -101939385.5%
Memory usage of dataframe is 100.22 MB
Memory usage of optimization is 102362528.00 MB
Decreased by -102133929.3%


,SK_ID_CURR,index,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Canceled_mean_high_risk,NAME_CONTRACT_STATUS_Canceled_mean_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,NUM_INSTALMENT_VERSION_min_low_risk,SK_ID_PREV_nunique_high_risk,SK_ID_PREV_nunique_low_risk,pred_1b,pred_1a,set1,set2a
0,100002,-1.732422,1.0,-0.577637,0.313232,-0.478027,-0.166138,-0.507324,-0.149780,1.506836,...,0.249878,0.0,-0.501953,0.582031,-0.627441,0.0,5.054688,0.986816,1.099609,0.860352
1,100003,-1.732422,0.0,-0.577637,0.933594,1.725586,0.592773,1.599609,-1.252930,-0.166870,...,0.249878,0.0,-0.501953,0.582031,-0.627441,0.0,-0.839844,-0.548340,-0.654785,-0.660156
2,100004,-1.732422,0.0,-0.577637,-0.927246,-1.153320,-1.404297,-1.091797,-0.783203,-0.689453,...,0.249878,0.0,-0.501953,0.582031,-0.627441,0.0,-0.334229,-0.507324,-0.353027,-0.508789
3,100006,-1.732422,0.0,-0.577637,-0.307129,-0.711426,0.177856,-0.653320,-0.928711,-0.680176,...,0.249878,0.0,-0.501953,0.582031,1.593750,0.0,-0.271240,-0.522949,-0.459717,-0.471680
4,100007,-1.732422,0.0,-0.577637,-0.431152,-0.213745,-0.361816,-0.068726,0.562988,-0.892578,...,0.249878,0.0,-0.501953,0.582031,-0.627441,0.0,-0.077087,-0.317627,-0.341064,-0.420654


In [56]:
df_test.drop('TARGET', axis = 1, inplace = True)
df_test.head()

,SK_ID_CURR,index,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Canceled_mean_high_risk,NAME_CONTRACT_STATUS_Canceled_mean_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,NUM_INSTALMENT_VERSION_min_low_risk,SK_ID_PREV_nunique_high_risk,SK_ID_PREV_nunique_low_risk,pred_1b,pred_1a,set1,set2a
0,100001,-1.732422,-0.577637,-0.307129,-0.075134,-0.451904,-0.239258,-0.146484,-0.734375,0.024017,...,0.249878,0.0,-0.501953,0.582031,-0.627441,0.0,-0.369385,-0.423584,-0.475586,-0.522949
1,100005,-1.732422,-0.577637,-0.638184,-0.935059,-0.671875,-0.970215,1.079102,-0.464355,-0.891113,...,0.249878,0.0,-0.501953,0.582031,-0.627441,0.0,0.969238,0.423096,0.289795,0.166748
2,100013,-1.732422,-0.577637,0.313232,0.159546,2.943359,0.247925,-0.127563,-0.916992,-0.886230,...,0.249878,0.0,1.992188,-1.718750,1.593750,0.0,-0.452637,-0.475830,-0.450439,-0.395996
3,100028,-1.732422,2.191406,1.346680,2.425781,1.511719,2.806641,0.399658,0.472168,0.221558,...,0.249878,0.0,1.992188,-1.718750,1.593750,0.0,-0.305664,-0.503418,-0.358887,-0.400146
4,100038,-1.732422,0.807129,0.106445,0.065796,0.342041,0.235718,-0.783203,0.686523,0.082153,...,0.249878,0.0,-0.501953,0.582031,-0.627441,0.0,1.168945,1.199219,0.448975,1.036133


In [57]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [58]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [59]:
train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246004, 522) (61502, 522)
[100]	training's auc: 0.790139	valid_1's auc: 0.778718
[200]	training's auc: 0.797806	valid_1's auc: 0.784872
[300]	training's auc: 0.803881	valid_1's auc: 0.788726
[400]	training's auc: 0.809164	valid_1's auc: 0.791397
[500]	training's auc: 0.813298	valid_1's auc: 0.79255
[600]	training's auc: 0.816895	valid_1's auc: 0.793326
[700]	training's auc: 0.820346	valid_1's auc: 0.79388
[800]	training's auc: 0.823336	valid_1's auc: 0.794145
[900]	training's auc: 0.826192	valid_1's auc: 0.794354
[1000]	training's auc: 0.828924	valid_1's auc: 0.794495
[1100]	training's auc: 0.831414	valid_1's auc: 0.794563
[1200]	training's auc: 0.833787	valid_1's auc: 0.794712
[1300]	training's auc: 0.836128	valid_1's auc: 0.794748
[1400]	training's auc: 0.838388	valid_1's auc: 0.794705
[1500]	training's auc: 0.840554	valid_1's auc: 0.794789
[1600]	training's auc: 0.842569	valid_1's auc: 0.794781
[1700]	training's auc: 0.844561	valid_1's au

[1400]	training's auc: 0.837883	valid_1's auc: 0.798612
[1500]	training's auc: 0.840005	valid_1's auc: 0.798602
[1600]	training's auc: 0.842029	valid_1's auc: 0.798508
[1700]	training's auc: 0.844032	valid_1's auc: 0.798411
[1800]	training's auc: 0.84598	valid_1's auc: 0.79833
[1900]	training's auc: 0.847917	valid_1's auc: 0.798231
[2000]	training's auc: 0.84981	valid_1's auc: 0.798141
[2100]	training's auc: 0.851626	valid_1's auc: 0.797981
[2200]	training's auc: 0.853479	valid_1's auc: 0.797901
[2300]	training's auc: 0.855219	valid_1's auc: 0.797786
[2400]	training's auc: 0.856897	valid_1's auc: 0.797737
[2500]	training's auc: 0.858589	valid_1's auc: 0.797604
[2600]	training's auc: 0.860195	valid_1's auc: 0.797481
[2700]	training's auc: 0.861834	valid_1's auc: 0.797398
[2800]	training's auc: 0.863428	valid_1's auc: 0.797331
[2900]	training's auc: 0.865006	valid_1's auc: 0.797283
[3000]	training's auc: 0.866442	valid_1's auc: 0.797226
[3100]	training's auc: 0.868	valid_1's auc: 0.79712

In [60]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
521,set2a,541.8,53.737324
520,set1,428.4,56.216546
518,pred_1a,413.8,83.265839
363,NEW_CREDIT_TO_ANNUITY_RATIO,400.6,16.712271
504,app_EXT_SOURCE_2_PER_DAYS_BIRTH,359.2,72.018748
478,USER_AGE,350.2,62.858571
225,DAYS_CREDIT_max,346.2,82.232597
144,APP_SCORE3_TO_BIRTH_RATIO,339.4,92.050530
263,EXT_SOURCE_3,327.8,88.046579
497,app_DAYS_EMPLOYED_PERC,327.4,68.108737


In [61]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [62]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [63]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_Kb_2-1.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.039667
1,100005,0.110220
2,100013,0.036859
3,100028,0.041531
4,100038,0.149409


In [64]:
###

In [65]:
baseline = Baseline('SK_ID_CURR')

sampler = optuna.samplers.TPESampler(seed=123)
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(objective, n_trials=50, n_jobs=5)

[I 2022-07-18 18:40:31,818] A new study created in memory with name: no-name-0e0d1312-60b1-40f1-b751-72e900824133
[I 2022-07-18 18:44:32,888] Trial 0 finished with value: 0.7947974003638432 and parameters: {'num_leaves': 164, 'min_child_samples': 105, 'min_sum_hessian_in_leaf': 5.835022530830985e-05, 'feature_fraction': 0.5386134805007059, 'bagging_fraction': 0.5078227655056979, 'lambda_l1': 21.820013417683985, 'lambda_l2': 0.3626949163414656}. Best is trial 0 with value: 0.7947974003638432.
[I 2022-07-18 18:44:33,820] Trial 1 finished with value: 0.7947974003638432 and parameters: {'num_leaves': 174, 'min_child_samples': 90, 'min_sum_hessian_in_leaf': 0.0014636022318094222, 'feature_fraction': 0.795671864608553, 'bagging_fraction': 0.6522824526055322, 'lambda_l1': 0.41839191229548656, 'lambda_l2': 2.9414779738812507}. Best is trial 0 with value: 0.7947974003638432.
[I 2022-07-18 18:44:41,087] Trial 3 finished with value: 0.7947974003638432 and parameters: {'num_leaves': 179, 'min_chil

[I 2022-07-18 19:01:27,967] Trial 21 finished with value: 0.7947974003638432 and parameters: {'num_leaves': 19, 'min_child_samples': 189, 'min_sum_hessian_in_leaf': 0.00012777298607384813, 'feature_fraction': 0.6579409538809956, 'bagging_fraction': 0.5059540228876593, 'lambda_l1': 18.685655253779643, 'lambda_l2': 0.2809412439403989}. Best is trial 0 with value: 0.7947974003638432.
[I 2022-07-18 19:02:02,482] Trial 22 finished with value: 0.7947974003638432 and parameters: {'num_leaves': 241, 'min_child_samples': 7, 'min_sum_hessian_in_leaf': 0.0001256796826211369, 'feature_fraction': 0.6814187957123742, 'bagging_fraction': 0.7810572059777074, 'lambda_l1': 8.17798843198447, 'lambda_l2': 0.05078861987283639}. Best is trial 0 with value: 0.7947974003638432.
[I 2022-07-18 19:02:10,130] Trial 23 finished with value: 0.7947974003638432 and parameters: {'num_leaves': 60, 'min_child_samples': 56, 'min_sum_hessian_in_leaf': 0.00013510013392515835, 'feature_fraction': 0.6910385175709406, 'baggin

[I 2022-07-18 19:19:10,574] Trial 42 finished with value: 0.7947974003638432 and parameters: {'num_leaves': 30, 'min_child_samples': 69, 'min_sum_hessian_in_leaf': 0.00018704612336334678, 'feature_fraction': 0.5030412018422482, 'bagging_fraction': 0.5078942542650281, 'lambda_l1': 8.499421614255807, 'lambda_l2': 0.13211650556514065}. Best is trial 0 with value: 0.7947974003638432.
[I 2022-07-18 19:19:30,519] Trial 43 finished with value: 0.7947974003638432 and parameters: {'num_leaves': 189, 'min_child_samples': 38, 'min_sum_hessian_in_leaf': 0.00016718502629174892, 'feature_fraction': 0.668681076063019, 'bagging_fraction': 0.5031838573591837, 'lambda_l1': 10.454102503481993, 'lambda_l2': 0.14548369552767176}. Best is trial 0 with value: 0.7947974003638432.
[I 2022-07-18 19:19:38,806] Trial 44 finished with value: 0.7947974003638432 and parameters: {'num_leaves': 184, 'min_child_samples': 37, 'min_sum_hessian_in_leaf': 0.0001774431750168082, 'feature_fraction': 0.6873176212533684, 'bagg

In [66]:
trial = study.best_trial
print("auc(best)={:.4f}".format(trial.value))
display(trial.params)

auc(best)=0.7948


{'num_leaves': 164,
 'min_child_samples': 105,
 'min_sum_hessian_in_leaf': 5.835022530830985e-05,
 'feature_fraction': 0.5386134805007059,
 'bagging_fraction': 0.5078227655056979,
 'lambda_l1': 21.820013417683985,
 'lambda_l2': 0.3626949163414656}

In [67]:
params_best = trial.params
params_best.update(params_base)
display(params_best)

{'num_leaves': 30,
 'min_child_samples': 70,
 'min_sum_hessian_in_leaf': 5.835022530830985e-05,
 'feature_fraction': 0.5386134805007059,
 'bagging_fraction': 0.5078227655056979,
 'lambda_l1': 21.820013417683985,
 'lambda_l2': 0.3626949163414656,
 'boosting_type': 'gbdt',
 'objective': 'binary',
 'metric': 'auc',
 'learning_rate': 0.02,
 'max_bin': 400,
 'max_depth': -1,
 'subsample': 1.0,
 'subsample_freq': 1,
 'colsample_bytree': 0.05,
 'min_split_gain': 0.5,
 'reg_alpha': 0.0,
 'reg_lambda': 100,
 'n_estimators': 10000}

In [68]:
train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5
                                    ,params=params_best)

-------------------- 0 --------------------
(246004, 522) (61502, 522)
[LightGBM] [Warning] feature_fraction is set=0.5386134805007059, colsample_bytree=0.05 will be ignored. Current value: feature_fraction=0.5386134805007059
[LightGBM] [Warning] lambda_l1 is set=21.820013417683985, reg_alpha=0.0 will be ignored. Current value: lambda_l1=21.820013417683985
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=5.835022530830985e-05, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=5.835022530830985e-05
[LightGBM] [Warning] bagging_fraction is set=0.5078227655056979, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5078227655056979
[LightGBM] [Warning] lambda_l2 is set=0.3626949163414656, reg_lambda=100 will be ignored. Current value: lambda_l2=0.3626949163414656
[100]	training's auc: 0.802746	valid_1's auc: 0.795533
[200]	training's auc: 0.806291	valid_1's auc: 0.796009
[300]	training's auc: 0.809795	valid_1's auc: 0.795874
[400]	training's au

[100]	training's auc: 0.802053	valid_1's auc: 0.798702
[200]	training's auc: 0.805566	valid_1's auc: 0.799032
[300]	training's auc: 0.809071	valid_1's auc: 0.798898
[400]	training's auc: 0.812515	valid_1's auc: 0.798641
[500]	training's auc: 0.815851	valid_1's auc: 0.798388
[600]	training's auc: 0.819028	valid_1's auc: 0.798162
[700]	training's auc: 0.822126	valid_1's auc: 0.797902
[800]	training's auc: 0.825175	valid_1's auc: 0.797644
[900]	training's auc: 0.828127	valid_1's auc: 0.797449
[1000]	training's auc: 0.830898	valid_1's auc: 0.797258
[1100]	training's auc: 0.833853	valid_1's auc: 0.797063
[1200]	training's auc: 0.836636	valid_1's auc: 0.796639
[1300]	training's auc: 0.839302	valid_1's auc: 0.796248
[1400]	training's auc: 0.841991	valid_1's auc: 0.796289
[1500]	training's auc: 0.844643	valid_1's auc: 0.796079
[1600]	training's auc: 0.847116	valid_1's auc: 0.795789
[1700]	training's auc: 0.849701	valid_1's auc: 0.795359
[1800]	training's auc: 0.852119	valid_1's auc: 0.795184
[

In [69]:
# 推論用データセットの作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

# predict
test_pred = baseline.predict_lgb(x_test
                       ,id_test
                       ,list_nfold=[0,1,2,3,4])

# make submission-file
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv("./home-credit-default-risk/submit/submission_Kb_2-2.csv", index=None)

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.
(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.034534
1,100005,0.111338
2,100013,0.041752
3,100028,0.042048
4,100038,0.170169
